In [154]:
def make_graph_dict(nodes, edges):
    G = {node: [] for node in nodes}
    for edge in edges:
        node1, node2 = edge[0], edge[1]
        G[node1].append(node2)
        G[node2].append(node1)
    return G

In [155]:
def make_directed_graph_dict(nodes, edges):
    G = {node: [] for node in nodes}
    for edge in edges:
        node1, node2 = edge[0], edge[1]
        G[node1].append(node2)
    return G

In [156]:
def dfs_recursive(G, start, dis=None, clos=None, par=None):
    if dis is None:
        dis = []
        clos = []
        par = dict()
    dis.append(start)
    for v in G[start]:
        if v not in dis:
            par[v] = start
            dfs_recursive(G, v, dis, clos, par)
    clos.append(start)
    return {"discovered": dis, "closed": clos, "parents": par}

In [86]:
def dfs_iterative(G, start, dis=set(), clos=[], par=dict(), comp = set()):
    stack = [(start, "open")]
    while stack:
        v, state = stack.pop()
        if state == "open":
            if v in dis:
                continue
            dis.add(v)
            comp.add(v)
            stack.append((v, "close"))
            chi = [i for i in G[v] if i not in dis][::-1]
            for i in chi:
                par[i] = v
                stack.append((i, "open"))
        else:
            clos.append(v)

In [158]:
def dfs_full(G, start=list(G.keys())[0]):
    dis = set()
    clos = []
    par = dict()
    comp = set()
    all_comp = set()
    dfs_iterative(G, start, dis, clos, par, comp)
    comp = frozenset(comp)
    all_comp.add(comp)
    for v in G:
        if v not in dis:
            comp = set()
            dfs_iterative(G, v, dis, clos, par, comp)
            comp = frozenset(comp)
            all_comp.add(comp)
    return {"discovered": dis, "closed": clos, "parents": par, "comps": all_comp}

In [159]:
def inverse_g(G):
    G_T = {v: [] for v in G.keys()}
    for v in G.keys():
        children = G[v]
        for j in children:
            G_T[j].append(v)
    return G_T

In [160]:
def scc(G):
    initial_dfs = dfs_full(G)
    G_ord = dict()
    for v in initial_dfs["closed"][::-1]:
        G_ord[v] = G[v]
    G_T = inverse_g(G_ord)
    second_dfs = dfs_full(G_T, start = list(G_T.keys())[0])
    sccs = second_dfs["comps"]
    return sccs

In [161]:
def ts(G, start):  # Topological sorting using DFS
    sorted_graph = dict()
    res = dfs_full(G, start)
    sort = res["closed"][::-1]
    for i in sort:
        sorted_graph[i] = G[i]
    return sorted_graph

In [4]:
def bfs_iterative(G, start = list(G.keys())[0], dis = set()):
    queue = [start]
    dis.add(start)
    temp_G_traversed = dict()
    for node in G:
        temp_G_traversed[node] = []
    while queue:
        local_root = queue[0]
        for v in G[local_root]:
            if v in dis:
                continue
            queue.append(v)
            dis.add(v)
            temp_G_traversed[local_root].append(v)
        G_traversed = {i: temp_G_traversed[i] for i in temp_G_traversed if temp_G_traversed[i]}
        queue = queue[1:]
    return {"Discovered": dis, "Traversed Graph": G_traversed}

def bfs_full(G, start=list(G.keys())[0]):
    dis = set()
    G_traversed = dict()
    temp_result = bfs_iterative(G, start, dis)
    dis = temp_result["Discovered"]
    G_traversed = temp_result["Traversed Graph"]
    for v in G:
        if v in dis:
            continue
        temp_result = bfs_iterative(G, v, dis)
        dis = temp_result["Discovered"]
        G_traversed = temp_result["Traversed Graph"] | G_traversed
    G_traversed = G_traversed | {j: [] for j in G if j not in G_traversed}
    return {"Discovered": dis, "Traversed Graph": G_traversed}

In [164]:
def fundamental_cycles(G, start):
    bfs_result = bfs_full(G, start)
    G_traversed = bfs_result["Traversed Graph"]
    fundamental_cycle_edges = {
        i: [j for j in G[i] if j not in G_traversed[i]] for i in G
    }
    return fundamental_cycle_edges

In [165]:
def weighted_G_to_edge_list(G_w):
    edge_list = []
    for v in G_w:
        for in_node in G_w[v]:
            temp_in_node_list = list(in_node)
            temp_in_node_list.insert(0, v)
            temp_edge_check = temp_in_node_list[:]
            temp_edge_check[0], temp_edge_check[1] = temp_edge_check[1], temp_edge_check[0]
            if temp_edge_check not in edge_list:
                edge_list.append(temp_in_node_list)
    return edge_list

def get_edge_weight(edge):
    return(edge[2])

def union(x, y, disjoint_sets):
    set_x = find(x, disjoint_sets)
    set_y = find(y, disjoint_sets)
    if set_x != set_y:
        disjoint_sets.remove(set_x)
        disjoint_sets.remove(set_y)
        disjoint_sets.append(set_x.union(set_y))
    return disjoint_sets

def find(x, disjoint_sets):
    for s in disjoint_sets:
        if x in s:
            return s
    else:
        return None

def kruskal_algorithm(G_w):
    G_edge_list = weighted_G_to_edge_list(G_w)
    G_edge_list.sort(key = get_edge_weight)
    min_span_tree = []
    node_disjoint_sets = [{i} for i in G_w]
    while len(node_disjoint_sets) > 1:
        for edge in G_edge_list:
            node1_set = find(edge[0], node_disjoint_sets)
            node2_set = find(edge[1], node_disjoint_sets)
            if node1_set != node2_set:
                node_disjoint_sets.remove(node1_set)
                node_disjoint_sets.remove(node2_set)
                node_disjoint_sets.append(node1_set.union(node2_set))
                min_span_tree.append(edge)
    G_span_tree = {v: [] for v in G_w}
    for edge in min_span_tree:
        G_span_tree[edge[0]].append((edge[1], edge[2]))
        G_span_tree[edge[1]].append((edge[0], edge[2]))
    return G_span_tree


In [166]:

import copy

def get_edge_weight(edge):
    return(edge[1])


def prim_algorithm(G_w, start_node):
    G_w_cp = copy.deepcopy(G_w)
    G_span_tree = {i: [] for i in G_w_cp}
    tree_nodes = {start_node}
    
    if start_node not in G_w_cp:
        return "Starting note non-existent in the graph adjacency list."
    for v in G_w_cp:
        G_w_cp[v].sort(key = get_edge_weight)

    for node in G_w_cp:
        G_w_cp[node] = [edge for edge in G_w_cp[node] if edge[0] != start_node]

    while len(tree_nodes) < len(G_w_cp):
        considered_edges = []
        for tree_node in tree_nodes:
            temp_edge = list(G_w_cp[tree_node][0])
            temp_edge.append(tree_node)
            considered_edges.append(temp_edge)
        considered_edges.sort(key = get_edge_weight)
        add_edge = considered_edges[0]
        add_node = add_edge[0]
        src_node = add_edge[2]
        G_span_tree[src_node].append((add_node, add_edge[1]))
        G_span_tree[add_node].append((src_node, add_edge[1]))
        tree_nodes.add(add_node)
        for node in G_w_cp:
            G_w_cp[node] = [edge for edge in G_w_cp[node] if edge[0] != add_node]

    return G_span_tree


In [169]:
def bellman_ford_algorithm(G_d_w, source_node):
    control_dict = {v: [[], float("inf"), None] for v in G_d_w} # Initiate a control dictionary noting the path to v, distance and predecessor node
    control_dict[source_node] = [[source_node], 0, source_node] # Initiate source node to have path of distance 0 (from itself to itself)
    discovered_nodes = {source_node}
    for i in range(0, len(G_d_w)):
        for from_node in G_d_w:
            for edge in G_d_w[from_node]:
                to_node = edge[0]
                if control_dict[from_node][1] + edge[1] < control_dict[to_node][1]:
                    new_path = control_dict[from_node][0][:]
                    new_path.append(to_node)
                    control_dict[to_node][0] = new_path
                    control_dict[to_node][1] = control_dict[from_node][1] + edge[1]
                    control_dict[to_node][2] = from_node
    for from_node in G_d_w:
        for edge in G_d_w[from_node]:
            to_node = edge[0]
            if control_dict[from_node][1] + edge[1] < control_dict[to_node][1]:
                print(f"A negative weight cycle exists with {from_node}")
                path_with_negative_cycle = control_dict[from_node][0]
                cycle_node = None
                cycle_path = [from_node]
                while cycle_node != from_node:
                    cycle_node = path_with_negative_cycle[:-1][-1]
                    path_with_negative_cycle = path_with_negative_cycle[:-1]
                    cycle_path.append(cycle_node)
                cycle_path = cycle_path[:0:-1]
                raise ValueError(f"Graph contains a negative cycle {cycle_path}")
    return control_dict


In [17]:
dictt = {1: "", 12: "qw"}
dictt.keys()
sett = set()

sett.update(list(dictt.keys()))
sett
dictt["newkey"] = []
dictt
sett = set()
sett.update(list(dictt.keys()))
list(sett)

[1, 'newkey', 12]

In [229]:
import copy

def initiate_s_t(G_d_c): # As defined in a transport network, initiate source and target vertices.
    G_d_c_copy = copy.deepcopy(G_d_c)
    G_d_c_copy['s'] = dict()
    non_source_nodes = set()
    for node in G_d_c_copy:
        non_source_nodes.update(list(G_d_c_copy[node].keys()))
    source_nodes = list({node for node in G_d_c_copy if node not in non_source_nodes and node != 's'})
    for node in source_nodes:
        G_d_c_copy['s'][node] = {'cap': float('inf'), 'flow': 0}
    target_nodes = [node for node in G_d_c_copy if G_d_c_copy[node] == {}]
    for node in target_nodes:
        G_d_c_copy[node]['t'] = {'cap': float('inf'), 'flow': 0}
    G_d_c_copy['t'] = dict()
    return G_d_c_copy
    

def make_reverse_edges(G_d_c_copy): # For Edmonds-Karp algorithm in a transport network with capacities, reverse edges should be made with capacities of 0 and flow of 0 for residual capacities further.
    for s_node in G_d_c_copy:
        for t_node in G_d_c_copy[s_node]:
            if s_node not in G_d_c_copy[t_node]:
                G_d_c_copy[t_node][s_node] = {'cap': 0, 'flow': 0}
    return G_d_c_copy

def bfs_iterative_ek(G, start = list(G.keys())[0], dis = set(), paths = [[list(G.keys())[0]]]):
    queue = [start]
    dis.add(start)
    temp_G_traversed = dict()
    for node in G:
        temp_G_traversed[node] = []
    
    while queue:
        local_root = queue[0]
        path_curr = [copy.deepcopy(path) for path in paths if path[-1] == local_root]
        for v in G[local_root]:
            if v in dis or G[local_root][v]['cap'] <= G[local_root][v]['flow']:
                continue
            path_curr_on_v = copy.deepcopy(path_curr)
            path_curr_on_v[0].append(v)
            paths.append(path_curr_on_v[0])
            paths = [path[:] for path in paths if path[-1] != local_root]
            queue.append(v)
            dis.add(v)
            temp_G_traversed[local_root].append(v)
        G_traversed = {i: temp_G_traversed[i] for i in temp_G_traversed if temp_G_traversed[i]}
        queue = queue[1:]
    return {"Discovered": dis, "Traversed Graph": G_traversed, "paths": paths}

def bfs_full_ek(G, start=list(G.keys())[0]):
    dis = set()
    G_traversed = dict()
    paths = [[start]]
    temp_result = bfs_iterative_ek(G, start, dis, paths)
    dis = temp_result["Discovered"]
    G_traversed = temp_result["Traversed Graph"]
    paths = temp_result['paths']
    G_traversed = G_traversed | {j: [] for j in G if j not in G_traversed}
    return {"Discovered": dis, "Traversed Graph": G_traversed, "paths": paths}

def edmonds_karp_algorithm(G_d_c):
    G_s_t = initiate_s_t(G_d_c)
    G_s_t_r = make_reverse_edges(G_s_t)
    print(G_s_t_r)
    min_df = float('inf')
    flow = 0
    used_path = 1
    while used_path: 
        bfs_to_t = bfs_full_ek(G_s_t_r, start = 's')
        local_paths = copy.deepcopy(bfs_to_t['paths'])
        used_path = next((path for path in local_paths if 't' in path), None)
        if used_path:
            for i in range(0, len(used_path) - 1):
                u, v = used_path[i], used_path[i + 1]
                edge = G_s_t_r[u][v]

                df = edge['cap'] - edge['flow']

                min_df = min(min_df, df)

            for i in range(0, len(used_path) - 1):
                u, v = used_path[i], used_path[i + 1]
                edge = G_s_t_r[u][v]
                rev_edge = G_s_t_r[v][u]

                edge['flow'] += min_df
                rev_edge['flow'] -= min_df

            flow += min_df
    
    return {'transport_network': G_s_t_r, 'flow': flow}

In [76]:
import copy

def gale_shapley_algorithm(G_pref):
    G_pref_copy = copy.deepcopy(G_pref)
    pairs = set()
    while len(pairs) != len(G_pref_copy["callers"]):
        for caller in G_pref_copy["callers"]:
            receiver = G_pref_copy["callers"][caller][0]
            current_rec_pair = next((pair for pair in pairs if receiver == pair[1]), None)
            if not current_rec_pair:
                pairs.add((caller, receiver))
            elif G_pref_copy["rec"][receiver].index(caller) < G_pref_copy["rec"][receiver].index(current_rec_pair[0]):
                pairs.remove(current_rec_pair)
                pairs.add((caller, receiver))
            else:
                pass
            G_pref_copy["callers"][caller] = G_pref_copy["callers"][caller][1:]
    return pairs

In [231]:
G = {5: [7], 6: [5], 7: [6], 1: [2, 3], 2: [1, 3], 3: [1, 2, 4], 4: [5, 3]}
G = {1: [2, 3], 3: [1, 2, 4], 2: [1, 3], 4: [3], 5: [6, 4], 6: [7], 7: [5]}
G = {"a": ["b", "c"], "r": ["a", "c"], "b": [], "c": []}
G = {1: [2], 2: [3], 3: [1]}
G = {1: []}
G = {1: [2], 2: [3], 3: [1], 4: [5], 5:[]}
G_d_c = {1: {2: {'cap': 20, 'flow': 0}, 50: {'cap': 999, 'flow': 0}},
          2: {3: {'cap': 10, 'flow': 0}, 4: {'cap': 5, 'flow': 0}},
            3: {},
              4: {},
              50: {}
              }
G_w = {1: [(2, 0)], 2: [(3, 500)], 3: [(1, 2)]}
G_w = {1: [(2, 0), (3, 2)], 2: [(3, 500), (1, 0)], 3: [(1, 2), (2, 500)]}
G_d_w = {1: [(2, 1)], 2: [(3, 2)], 3: [(4, 2)], 4: [(5, 3)], 5: []}
G_pref = {"callers": {"a": [2, 3, 1], "b": [2, 1, 3], "c": [3, 2, 1]}, "rec": {1: ["c", "b", "a"], 2: ["b", "a", "c"], 3: ["c", "b", "a"]}}
G_edge_list = [(1, 2, 0), (2, 3, 500), (3, 1, 2)]
root = 1

In [ ]:
res_dfs_r = dfs_recursive(G, root)
res_dfs_f = dfs_full(G, root)
print(res_dfs_r)
print(res_dfs_f)

In [ ]:
topological_sort = ts(G, root)
print(topological_sort)
strong_components = scc(G)
print(strong_components)

In [ ]:
res_bfs_f = bfs_full(G, root)
print(res_bfs_f)
res_fund_cycl = fundamental_cycles(G, root)
print(res_fund_cycl)

In [172]:
res_kruskal = kruskal_algorithm(G_w)
print(res_kruskal)

{1: [(2, 0), (3, 2)], 2: [(1, 0)], 3: [(1, 2)]}


In [173]:
res_prim = prim_algorithm(G_w, 1)
print(res_prim)

{1: [(2, 0), (3, 2)], 2: [(1, 0)], 3: [(1, 2)]}


In [174]:
res_bellman_ford = bellman_ford_algorithm(G_d_w, 1)
print(res_bellman_ford)

{1: [[1], 0, 1], 2: [[1, 2], 1, 1], 3: [[1, 2, 3], 3, 2], 4: [[1, 2, 3, 4], 5, 3], 5: [[1, 2, 3, 4, 5], 8, 4]}


In [ ]:
res_ek = edmonds_karp_algorithm(G_d_c)
print(res_ek)

In [ ]:
res_gs = gale_shapley_algorithm(G_pref)
print(res_gs)